Goal is to predict category of an incident (from temporal and gegraphical information) Type: Classification

# Imports

In [ ]:
!git clone https://github.com/Pink-Raccoon/MLDM_KaggleProjekt.git

In [ ]:
#!pip install geopandas
#!pip install osmnx # https://pygis.io/docs/d_access_osm.html
#!pip install arcgis # https://developers.arcgis.com/python/guide/part5-reverse-geocoding/#the-simple-reverse-geocode-examples

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import folium as fo
import seaborn as sns
# import osmnx
# import osmnx as ox
# import geopandas as gpd

#Loading data from file

In [ ]:
train_data = pd.read_csv("/content/MLDM_KaggleProjekt/sf-crime/train.csv.zip")
train_data.columns

RANDOM_SEED = 0xdeadbeef
train_data.shape
train_data.head(3)

In [ ]:
test_data = pd.read_csv("/content/MLDM_KaggleProjekt/sf-crime/test.csv.zip")
test_data.head(3)

# Data-Processing

In [ ]:
#Descript, Resoultion and address are not needed for analysis so they will be dropped
train_data = train_data.drop(columns=['Descript','Resolution', 'Address'])
train_data.dropna(inplace=True)


In [ ]:
test_data = test_data.drop(columns=['Address'])
test_data.dropna(inplace=True)

In [ ]:
train_data.dtypes

In [ ]:
#Change datatype of time 
date = pd.to_datetime(train_data["Dates"])
train_data = train_data.drop(columns=['DayOfWeek'])
train_data = train_data.drop(columns=['Dates'])
train_data["Year"] = date.dt.year
train_data["Month"] = date.dt.month
train_data["Day"] = date.dt.day
train_data["Hour"] = date.dt.hour
train_data["DayOfWeek"] = date.dt.weekday #monday = 0, tuesday = 1, ...
train_data.dropna(inplace=True)

In [ ]:
date = pd.to_datetime(test_data['Dates'])
test_data = test_data.drop(columns=['DayOfWeek'])
test_data = test_data.drop(columns=['Dates'])
test_data["Year"] = date.dt.year
test_data["Month"] = date.dt.month
test_data["Day"] = date.dt.day
test_data["Hour"] = date.dt.hour
test_data["DayOfWeek"] = date.dt.weekday  #monday = 0, tuesday = 1, ...
test_data.dropna(inplace=True)

In [ ]:
train_data.shape

In [ ]:
#plt.scatter(train_data.X, train_data.Y)
#coordinate = np.stack((train_data.X, train_data.Y), axis=1)
#plt.scatter(coordinate[:, 0], coordinate[:, 1])


In [ ]:
outliers_train = train_data[train_data["Y"] > 40.] 

In [ ]:
#outliers_train.sort_values(by='Category')

In [ ]:
train_data = train_data[train_data["Y"] < 40.] 
train_data.drop_duplicates()
print(train_data.shape)

In [ ]:
#plt.scatter(test_data.X, test_data.Y)

filtered_coordinate = test_data.Y > 40
new_coord = test_data.Y[filtered_coordinate]
print(new_coord.shape)

print(test_data.shape)
test_data = test_data[test_data["Y"] < 40.] 
print(test_data.shape)


In [ ]:
#plt.scatter(train_data.X, train_data.Y)

In [ ]:
#plt.scatter(test_data.X, test_data.Y)

In [ ]:
#perform data binning on points variable
bin_labels = [1, 2, 3, 4, 5, 6]
df_train_bin = train_data
df_train_bin['Hour_bin'] = pd.qcut(df_train_bin['Hour'], q=6, labels=bin_labels)

#view updated DataFrame
df_train_bin['Hour_bin'].value_counts()
df_train_bin = df_train_bin.drop(columns=['Hour'])

#(-0.001, 7.0]	early morning
#(7.0, 11.0]	morning
#(11.0, 14.0]	noon
#(14.0, 17.0]	afternoon
#(17.0, 20.0]	evening  
#(20.0, 23.0]	night

In [ ]:
#df_train_bin

In [ ]:
df_test_bin = test_data
df_test_bin['Hour_bin'] = pd.qcut(df_test_bin['Hour'], q=6)

#view updated DataFrame
df_test_bin = df_test_bin.drop(columns=['Hour'])
df_test_bin['Hour_bin'].value_counts()

In [ ]:
#df_test_bin

In [ ]:
#perform data binning on points variable
bin_labels2 = np.arange(79)
bin_labels3 = np.arange(78)

#perform data binning on points variable

df_train_bin['coordX'] = pd.qcut(df_train_bin['X'], q=80, labels=bin_labels2, duplicates='drop')
df_train_bin['coordY'] = pd.qcut(df_train_bin['Y'], q=80, labels=bin_labels3, duplicates='drop')


#view updated DataFrame
df_train_bin['coordX'].value_counts()
df_train_bin['coordY'].value_counts()


In [ ]:
df_train_bin = df_train_bin.drop(columns=['X'])
df_train_bin = df_train_bin.drop(columns=['Y'])

In [ ]:
#perform data binning on points variable
bin_labels2 = np.arange(79)
bin_labels3 = np.arange(78)

#perform data binning on points variable

df_test_bin['coordX'] = pd.qcut(df_test_bin['X'], q=80, labels=bin_labels2, duplicates='drop')
df_test_bin['coordY'] = pd.qcut(df_test_bin['Y'], q=80, labels=bin_labels3, duplicates='drop')


#view updated DataFrame
df_test_bin['coordX'].value_counts()
df_test_bin = df_test_bin.drop(columns=['X'])
df_test_bin['coordY'].value_counts()
df_test_bin = df_test_bin.drop(columns=['Y'])

#Data splitten

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 


In [ ]:
# Python3 program to Convert a
# list to dictionary
def addKey(i,res_dct,lst):
  res_dct[lst[i]] = i
  return res_dct
def Convert(lst,res_dct):
  for i in range(0, len(lst)):
    res_dct=addKey(i,res_dct,lst)
  return res_dct

In [ ]:
res_dct = {}
df = pd.DataFrame(df_train_bin,columns=['Category'])
cat_arr = np.unique(df.to_numpy())
cat_dict = Convert(cat_arr,res_dct)
print(cat_dict)
print(df.to_numpy()[0])

In [ ]:
res2_dct = {}
df2 = pd.DataFrame(df_train_bin,columns=['PdDistrict'])
dist_arr = np.unique(df2.to_numpy())
dist_dict = Convert(dist_arr,res2_dct)
print(dist_dict)


In [ ]:
target = df.Category.apply(lambda category: cat_dict[category])
df_train_bin['Districts'] = df2.PdDistrict.apply(lambda PdDistrict: dist_dict[PdDistrict])

In [ ]:
res2_dct = {}
df2 = pd.DataFrame(df_test_bin,columns=['PdDistrict'])
dist_arr = np.unique(df2.to_numpy())
dist_dict = Convert(dist_arr,res2_dct)
print(dist_dict)

In [ ]:
target = df.Category.apply(lambda category: cat_dict[category])
df_test_bin['Districts'] = df2.PdDistrict.apply(lambda PdDistrict: dist_dict[PdDistrict])

In [ ]:
print(df_train_bin.shape)
print(target.shape)

In [ ]:
df_train_bin.head(2)

In [ ]:
target[:2]

In [ ]:
df_train_bin = df_train_bin.drop(columns=['Category'])
df_train_bin.dropna(inplace=True)
df_train_bin = df_train_bin.drop(columns=['PdDistrict'])
df_train_bin.dropna(inplace=True)

In [ ]:
X_train, X_valid, y_train, y_valid= train_test_split(df_train_bin,target,train_size=0.8, random_state=RANDOM_SEED)

In [ ]:
print(X_train.shape)
print(X_valid.shape)

In [ ]:
print(X_train.shape)
print(y_train.shape)

# NN

In [ ]:
X_train.head(1)

In [ ]:
# here we import everything you need to build the CNN
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping

In [ ]:
def plot_history(history: keras.callbacks.History):
  """
  plot the training and validation loss for each training epoch

  history: a History object, output of the .fit method of a keras model
  """
  n = len(history.history['loss'])
  plt.plot(np.arange(n), history.history['loss'], label="training loss")
  plt.plot(np.arange(n), history.history['val_loss'], label="validation loss")
  plt.xticks(range(0, n + 1, 2))
  plt.legend()
  plt.show()

In [ ]:
X_train_ = X_train.astype('float32') 
X_valid = X_valid.astype('float32') 
y_train = keras.utils.to_categorical(y_train)
y_valid = keras.utils.to_categorical(y_valid)

print("train samples: ", X_train.shape[0])
print("test samples: ", X_valid.shape[0])

print("shape of one sample: ", X_train.head(1).shape)

In [ ]:
class_num = 8
nn_model = Sequential()
nn_model.add(Input(shape=(8)))
#model.add(Input(shape=(X_train.head(1).shape)))
nn_model.add(Dense(class_num * 25, use_bias=True, activation="relu"))
nn_model.add(Dense(class_num * 20, use_bias=True, activation= keras.activations.relu))
nn_model.add(Dropout(rate=0.15))
nn_model.add(Dense(class_num * 15, use_bias=True, activation="relu"))
nn_model.add(Dropout(rate=0.1))
nn_model.add(Dense(class_num * 10, use_bias=True, activation="relu"))
nn_model.add(Dense(39, activation=keras.activations.softmax))

nn_model.summary()

In [ ]:
nn_model.compile(
    loss=keras.losses.CategoricalCrossentropy(),
    optimizer=keras.optimizers.Adamax(learning_rate=0.1),
    metrics=["accuracy"])

early_stopping = EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=10)

# note that we pass the early_stopping object as a callback here
history = nn_model.fit(x= X_train, y=y_train,
  epochs=80,
  batch_size=640S,
  validation_split=.10,
  callbacks=[early_stopping]
)


In [ ]:
plot_history(history)

#Plotting the Data

In [ ]:
train_data['Category'].value_counts().plot(kind='barh',figsize=(30, 15))

In [ ]:
plt.figure(figsize=(15,8))
train_data['Hour'].value_counts().plot.pie(autopct='%1.1f%%')
plt.title('Hour')

In [ ]:
plt.figure(figsize=(15,8))
train_data['DayOfWeek'].value_counts().plot.pie(autopct='%1.1f%%')
plt.title('DayOfWeekay')

In [ ]:
plt.figure(figsize=(15,8))
train_data['Year'].value_counts().plot.pie(autopct='%1.1f%%')
plt.title('Year')

In [ ]:
plt.figure(figsize=(15,8))
train_data['PdDistrict'].value_counts().plot.pie(autopct='%1.1f%%')
plt.title('PdDistrict')

In [ ]:
#train_data.groupby('Category')['Hour'].value_counts()
bins_stuff = [2,3,4,5, 6,7,8,9, 10,11,12,13, 14,15,16,17, 18,19,20,21, 22,23,0,1]
#grps = train_data.groupby(['Category',pd.cut(train_data.Hour,bins_stuff)])
#grps.size().unstack()


In [ ]:
#train_data['Category'].hist(by=train_data['Hour'],figsize=(200,100))
train_data['Hour'] = pd.cut(train_data.Hour, bins = range(0,24),right=False)
(train_data.groupby('Hour').Category.value_counts().unstack().plot.bar(width=1,stacked=True, figsize=(30,30)))

In [ ]:
bins_stuff = [2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015]
grps = train_data.groupby(['Category',pd.cut(train_data.Year,bins_stuff)])
grps.size().unstack()

In [ ]:
bins_stuff = [1,2,3,4,5,6,7]
grps = train_data.groupby(['Category',pd.cut(train_data.Day,bins_stuff)])
grps.size().unstack()

In [ ]:
m = fo.Map(location=[37.773972,-122.446747],zoom_start=13)
fo.Marker(
    location=[37.774599,-122.425892],
    popup="Northern",
    icon=fo.Icon(color="darkgreen"),
).add_to(m)
fo.Marker(
    location=[37.771541,-122.438738],
    popup="Park",
    icon=fo.Icon(color="lightgreen"),
).add_to(m)
fo.Marker(
    location=[37.714056,-122.459033],
    popup="Taraval",
    icon=fo.Icon(color="gray"),
).add_to(m)
fo.Marker(
    location=[37.731948,-122.447364],
    popup="Ingleside",
    icon=fo.Icon(color="pink"),
).add_to(m)
fo.Marker(
    location=[37.780266,-122.403390],
    popup="Southern",
    icon=fo.Icon(color="darkblue"),
).add_to(m)
fo.Marker(
    location=[37.738212,-122.394926],
    popup="Bayview",
    icon=fo.Icon(color="red"),
).add_to(m)
fo.Marker(
    location=[37.78393202772729,-122.412597377187],
    popup="Tenderloin",
    icon=fo.Icon(color="black"),
).add_to(m)
fo.Marker(
    location=[37.7487740050495,-122.42257233421302],
    popup="Mission",
    icon=fo.Icon(color="orange"),
).add_to(m)

fo.Marker(
    location=[37.789434763004	,-122.406958660602],
    popup="Central",
    icon=fo.Icon(color="purple"),
).add_to(m)	
fo.Marker(
    location=[37.7725405391593	,-122.466204667893],
    popup="Richmond",
    icon=fo.Icon(color="lightblue"),
).add_to(m)	

m


In [ ]:
# List key-value pairs for tags
tags = {'building': True}   
#buildings = ox.geometries_from_place("Piedmont, California, USA", tags)
#buildings = ox.geometries_from_point(center_point=point, tags=tags)
#buildings.head()

In [ ]:
#point = (48.873446, 2.294255)
#dist = 612
#gdf = ox.geometries_from_point(point, tags, dist=dist)
#gdf.head()

# svm

In [ ]:
def plot_svc_decision_function(model, X, y):
    """
    Plot the decision function for a 2D SVC

    model: a fitted SVM model of type sklearn.svm.LinearSVC or sklearn.svm.SVC
    X: a 2D numpy array of shape [n_samples, 2], containing data points to be classified
    y: a 1D numpy array of shape [n_samples], containing the labels for the data points, labels should be 0 or 1
    """
    plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')

    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

In [ ]:
from sklearn.svm import LinearSVC
model = LinearSVC() # initialize the model
model.fit(X_train,y_train) # fit the model = learn the decision boundaries

plot_svc_decision_function(model, X_train,y_train)